In [193]:
import pandas as pd
from functools import reduce
import json , pandas as pd, numpy as np , networkx as nx 
import ast
import spotipy

In [226]:
file_path = 'sentiment_data.pkl'
df = pd.read_pickle(file_path)
df.count()

track_name           24985
track_id             24985
explicit             24985
artist_names         24985
artist_ids           24985
acousticness         24985
danceability         24985
energy               24985
instrumentalness     24985
liveness             24985
loudness             24985
speechiness          24985
tempo                24985
popular_artist       24985
popular_artist_id    24985
combined_genres      24985
lyrics               19066
sentiments           24985
dtype: int64

In [9]:
df.columns

Index(['track_name', 'track_id', 'track_number', 'disc_number', 'duration_ms',
       'explicit', 'popularity', 'preview_url', 'isrc', 'album_name',
       'album_id', 'album_type', 'album_total_tracks', 'album_release_date',
       'album_release_date_precision', 'album_images', 'artist_names',
       'artist_ids', 'external_url', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'popular_artist',
       'popular_artist_id', 'combined_genres', 'combined_popularity',
       'combined_followers', 'artist_popularity', 'artist_followers',
       'video_id', 'lyrics'],
      dtype='object')

In [222]:
required_cols = ["track_name" , 'track_id', 'explicit', 'artist_names',
       'artist_ids', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness',
       'speechiness', 'tempo', 'popular_artist',
       'popular_artist_id', 'combined_genres', 'lyrics']
df = df.loc[: ,required_cols]
df.head()

,track_name,track_id,explicit,artist_names,artist_ids,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,popular_artist,popular_artist_id,combined_genres,lyrics
0,Shotgun,4ofwffwvvnbSkrMSCKQDaC,False,[George Ezra],[2ysnwxxNtSgbb9t1m2Ur4j],0.286,0.673,0.735,0.00000,0.242,-4.733,0.0457,115.744,George Ezra,2ysnwxxNtSgbb9t1m2Ur4j,"[folk-pop, neo-singer-songwriter]",Homegrown alligator\nSee you later\nGotta hit ...
1,Miravi,2ssvA9HSGxaWAek1S2afbH,False,[Soli (Soliman Vaseghi)],[20JGDHGeqDV9HaaoDpUFQK],0.878,0.460,0.590,0.00000,0.230,-12.732,0.0346,154.924,Soli (Soliman Vaseghi),20JGDHGeqDV9HaaoDpUFQK,[],حال که دیوانه شدم ، می روی\nبی سر و سامانه شدم...
2,Bello Belisimmo,0khhnIB1aqU6emDCXONkQD,True,"[Krisko, Homiez Clique]","[4oopAFtLdKXCoq2XoEjDZD, 00eWmLCL4w3ISciGAydSS4]",0.149,0.958,0.618,0.00000,0.034,-8.786,0.3310,104.061,Krisko,4oopAFtLdKXCoq2XoEjDZD,"[bulgarian hip hop, bulgarian pop, chalga]",Кво стаа София? (yea)\nСтолични наркотици (ай)...
3,ELLE SAIT,5erH8ainax63isS82Ahoty,True,[kobzx2z],[1e1A7s86U6f7cyJAivpfY3],0.212,0.642,0.595,0.00508,0.156,-8.324,0.0315,98.027,kobzx2z,1e1A7s86U6f7cyJAivpfY3,[speed up turkce],"Elle sait que j'm'en remettrai jamais, j'm'en ..."
4,Chega,01mw8dztHKWqzhCySgKllc,False,[Eduardo Costa],[2OUgs5INWK2UEvZ6amWiJr],0.447,0.690,0.731,0.00000,0.343,-5.080,0.0353,110.083,Eduardo Costa,2OUgs5INWK2UEvZ6amWiJr,"[sertanejo pop, sertanejo universitario, serta...",Chega\r\nDe viver humilhado tentando mostrar-t...


In [77]:
class GenreRecommender():
    def __init__(self):
        self.song_data_filepath = "song_data/filtered_songdata.json"
        with open(self.song_data_filepath,'r',encoding='utf-8') as f: self.song_data = json.loads(f.read())

        self.song_genre_dict = { d['SongID'] : list(set(d['SongData']['genre']+d['SongData']['artists'][0]['genre'])) for d in self.song_data}
        self.songs = sorted(list(set([x['SongID'] for x in self.song_data])))
        self.genres = sorted(list(set(reduce(lambda x,y: x+y , [x['SongData']['genre'] + x['SongData']['artists'][0]['genre'] for x in self.song_data]))))
        self.num_songs = len(self.songs)
        self.num_genres = len(self.genres)

        self.graph = nx.Graph()
        self.graph.add_nodes_from(self.songs)
        self.graph.add_nodes_from(self.genres)
        for a,b in self.song_genre_dict.items():
            for g in b:
                self.graph.add_edge(a,g)
    def genreSim(self,songid1 , songid2):
        a1 = set(self.graph.neighbors(songid1))
        a2 = set(self.graph.neighbors(songid2))
        return 1 - np.exp(-3/2*len(a1.intersection(a2)))
genre_rec = GenreRecommender()

In [198]:
id1 = "0VjIjW4GlUZAMYd2vXMi3b"  
id2 = "1bDbXMyjaUIooNwFE9wn0N"
genre_sim = genre_rec.genreSim(id1, id1)
print(genre_sim)

0.9888910034617577


In [ ]:
def similarity_genre_only (id1 , id2) : 
    genre_sim = genre_rec.genreLoss(id1, id1)
    return genre_sim

def similarity_whole_row (id1 , id2) : 
    
    genre_sim = genre_rec.genreLoss(id1, id1)

In [187]:
class ArtistRecommender():
    def __init__(self,song_data_filepath):
        self.song_data_filepath = song_data_filepath
        with open(self.song_data_filepath,'r',encoding='utf-8') as f: self.song_data = json.loads(f.read())
        self.song_artist_dict = { d['SongID'] : [artistdet['id'] for artistdet in d['SongData']['artists']] for d in self.song_data}
        self.artist_ids = set(reduce(lambda x,y:x+y , [[artistdet['id'] for artistdet in d['SongData']['artists']] for d in self.song_data]))
        self.artist_relations = None

    def getArtistRelationFromSpotify(self,spotify_creds,savefilename):
        sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(spotify_creds[0],spotify_creds[1]))
        with open(savefilename,'r+',encoding='utf-8') as filesaver : self.edgelist = json.loads(filesaver.read())['edges']
        print("EdgeList loaded : ", len(self.edgelist))
        temp_edges = []
        for idx , artistid in enumerate(self.artist_ids):
            print(f"Sn. {idx+1} ,{artistid:24}", end=" ")
            while True:
                data = sp.artist_related_artists(artist_id=artistid)
                if 'error' in data.keys():
                    print(data,flush=True)
                    print(data['error']['status'],flush=True)
                    time.sleep(3)
                    continue
                filtered_data = [d['id'] for d in data['artists'] if d['id'] in self.artist_ids]
                temp_edges += list(map(lambda x: sorted([artistid,x]) , filtered_data))
                print(f"{len(filtered_data)} entries ", filtered_data )
                if (idx%10 == 0):
                    self.edgelist = self.edgelist + temp_edges
                    temp_edges = []
                    with open(savefilename,'w',encoding='utf-8') as f: f.write(json.dumps({'edges':self.edgelist}))
                break  
        self.edgelist += temp_edges
        with open(savefilename,'w',encoding='utf-8') as f: f.write(json.dumps({'edges':self.edgelist}))
        

    def getArtistRelationFromFile(self,filename):
        with open(filename,'r',encoding='utf-8') as f: self.artist_relations = json.loads(f.read())
        self.graph = nx.Graph()
        self.graph.add_nodes_from(self.artist_ids)
        self.graph.add_edges_from(self.artist_relations['edges'])

    def artistSim(self,artistid1,artistid2):
        translate = lambda x : np.exp(-x/6)
        if self.graph.has_node(artistid1) and self.graph.has_node(artistid2) and nx.has_path(self.graph,artistid1,artistid2) :
            return translate(nx.shortest_path_length(self.graph,artistid1,artistid2))
        else: return 0

artist_rec = ArtistRecommender('song_data/filtered_songdata.json')
artist_rec.getArtistRelationFromFile('song_data/artist_edges.json')
e1 = '6eUKZXaKkcviH0Ku9w2n3V'
e2 = '6eUKZXaKkcviH0Ku9w2n3V'
artist_rec.artistSim(e1,e2)

1.0

In [188]:
e1 = '6eUKZXaKkcviH0Ku9w2n3V'
e2 = "2OdNeYsVpZWzIKREJi72NI"
print(artist_rec.artistSim(e1,e2))

0


In [69]:
file_path = 'sentiment_data.pkl'
df = pd.read_pickle(file_path)

In [207]:
for index , row in df[2:].iterrows() : 
    print(row)
    if index == 3 : 
        break


track_name                                             Bello Belisimmo
track_id                                        0khhnIB1aqU6emDCXONkQD
explicit                                                          True
artist_names                                   [Krisko, Homiez Clique]
artist_ids            [4oopAFtLdKXCoq2XoEjDZD, 00eWmLCL4w3ISciGAydSS4]
acousticness                                                     0.149
danceability                                                     0.958
energy                                                           0.618
instrumentalness                                                   0.0
liveness                                                         0.034
loudness                                                        -8.786
speechiness                                                      0.331
tempo                                                          104.061
popular_artist                                                  Krisko
popula

In [208]:
df.head()

,track_name,track_id,explicit,artist_names,artist_ids,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,popular_artist,popular_artist_id,combined_genres,lyrics,sentiments
0,Shotgun,4ofwffwvvnbSkrMSCKQDaC,False,[George Ezra],[2ysnwxxNtSgbb9t1m2Ur4j],0.286,0.673,0.735,0.00000,0.242,-4.733,0.0457,115.744,George Ezra,2ysnwxxNtSgbb9t1m2Ur4j,"[folk-pop, neo-singer-songwriter]",Homegrown alligator\nSee you later\nGotta hit ...,"[0.062261879444122314, 0.10762041062116623, 0...."
1,Miravi,2ssvA9HSGxaWAek1S2afbH,False,[Soli (Soliman Vaseghi)],[20JGDHGeqDV9HaaoDpUFQK],0.878,0.460,0.590,0.00000,0.230,-12.732,0.0346,154.924,Soli (Soliman Vaseghi),20JGDHGeqDV9HaaoDpUFQK,[],حال که دیوانه شدم ، می روی\nبی سر و سامانه شدم...,"[0.11398503929376602, 0.13223107159137726, 0.2..."
2,Bello Belisimmo,0khhnIB1aqU6emDCXONkQD,True,"[Krisko, Homiez Clique]","[4oopAFtLdKXCoq2XoEjDZD, 00eWmLCL4w3ISciGAydSS4]",0.149,0.958,0.618,0.00000,0.034,-8.786,0.3310,104.061,Krisko,4oopAFtLdKXCoq2XoEjDZD,"[bulgarian hip hop, bulgarian pop, chalga]",Кво стаа София? (yea)\nСтолични наркотици (ай)...,"[0.06204673647880554, 0.0931626558303833, 0.28..."
3,ELLE SAIT,5erH8ainax63isS82Ahoty,True,[kobzx2z],[1e1A7s86U6f7cyJAivpfY3],0.212,0.642,0.595,0.00508,0.156,-8.324,0.0315,98.027,kobzx2z,1e1A7s86U6f7cyJAivpfY3,[speed up turkce],"Elle sait que j'm'en remettrai jamais, j'm'en ...","[0.02467518486082554, 0.012875158339738846, 0...."
4,Chega,01mw8dztHKWqzhCySgKllc,False,[Eduardo Costa],[2OUgs5INWK2UEvZ6amWiJr],0.447,0.690,0.731,0.00000,0.343,-5.080,0.0353,110.083,Eduardo Costa,2OUgs5INWK2UEvZ6amWiJr,"[sertanejo pop, sertanejo universitario, serta...",Chega\r\nDe viver humilhado tentando mostrar-t...,"[0.11756470054388046, 0.11561193317174911, 0.1..."


In [103]:
print(row2.loc[: ,"popular_artist_id"].item())
print(row1.loc[: ,"popular_artist_id"].item())

2OdNeYsVpZWzIKREJi72NI
0WwSkZ7LtFUFjGjMZBMt6T


In [206]:
row1

,track_name,track_id,explicit,artist_names,artist_ids,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,popular_artist,popular_artist_id,combined_genres,lyrics,sentiments
23233,Erika,18gsp3kqbBD3SDvm364cqI,True,[Young Adult],[0CrWiu9wRvXhkmw6JVF6o2],0.698,0.592,0.67,0.000067,0.154,-11.216,0.0452,114.169,Young Adult,0CrWiu9wRvXhkmw6JVF6o2,[meme],Now what am I supposed to do\r\nWhen I want yo...,NaN


In [221]:
def individual_index_similarity(vec1, vec2):
    similarities = [abs(a - b) / max(abs(a), abs(b)) if max(abs(a), abs(b)) != 0 else 1 for a, b in zip(vec1, vec2)]
    return np.mean(similarities)

def get_row_by_id(track_id):
    return df[df['track_id'] == track_id]

def cos_sim_sentim (vec1 , vec2) : 
    return np.dot(vec1 , vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    
def senti_list (row) : 
    sentiment = row.loc[: ,"sentiments"].item()
    return ast.literal_eval(sentiment)

def get_sp_analysis_feat(row): 
    cols_to_consider = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo']
    return [row.loc[: , i].item() for i in cols_to_consider]


In [ ]:
def final_sim_func(id1 , id2) : 
    senti+nt = 0
    

    row1 = get_row_by_id(id1)
    row2 = get_row_by_id(id2)
    
    artist_sim = artist_rec.artistSim(row1.loc[: ,"popular_artist_id"].item(),row2.loc[: ,"popular_artist_id"].item())
    genre_sim = genre_rec.genreSim(id1, id2)

    if sentiment_1 == str([0,0,0,0,0,0,0])
    sentiment_1 = senti_list (row1)
    sentiment_2 = senti_list (row2)
    sent_sim = cos_sim_sentim(sentiment_1 , sentiment_2)

    analysis_feat_1 = get_sp_analysis_feat(row1)
    analysis_feat_2 = get_sp_analysis_feat(row2)

    sp_audio_feat_sim = individual_index_similarity(analysis_feat_1 , analysis_feat_2)

    weighted_metric = 

    

In [209]:
id1 = "4yqtwO7MQIIXqoiRBPHAgR"
id1 = "4ofwffwvvnbSkrMSCKQDaC"
id2 = "01mw8dztHKWqzhCySgKllc"

row1 = df[df["track_id"] == id1]
row2 = df[df["track_id"] == id2]

artist_sim = artist_rec.artistSim(row1.loc[: ,"popular_artist_id"].item(),row2.loc[: ,"popular_artist_id"].item())
print("artist sim :", artist_sim)

genre_sim = genre_rec.genreSim(id1, id2)
print(genre_sim)


artist sim : 0.18887560283756183
0.0


In [214]:
sentiment_1 = senti_list (row1)
sentiment_2 = senti_list (row2)

sent_sim = cos_sim_sentim(sentiment_1 , sentiment_2)
print(sent_sim)

0.6151294909778078


In [219]:

analysis_feat_1 = get_sp_analysis_feat(row1)
analysis_feat_2 = get_sp_analysis_feat(row2)

individual_index_similarity(analysis_feat_1 , analysis_feat_2)

0.25368841695482813

In [67]:
# # Initialize Transformers pipeline and tokenizer
# pipe = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", device=0 , top_k = 7)
# tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")

# def sentiments (lrcs) : 
#     tokens = tokenizer(lrcs, truncation=True, max_length=512, return_tensors="pt")
#     truncated_lrcs = tokenizer.decode(tokens.input_ids[0], skip_special_tokens=True)
#     output = pipe(truncated_lrcs)[0]
#     scores = {i["label"] : i["score"] for i in output}
#     lst = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
#     sentiments = [scores.get(key, 0) for key in lst]
#     return sentiments

# sentiments(lrcs)